<a href="https://colab.research.google.com/github/carlosemiliorabazo/deepLearning/blob/master/2_3_Engranajes_RNA_Operaciones_con_Tensores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Los Engranajes de las Redes Neuronales: Operaciones con Tensores

**Nota:** No vamos a usar notación matemática algebráica de Tensores, dado que no es el objetivo de esta sección. Necesitamos su conocimiento conceptual y práctico

Cuando al princípio creabamos la RNA con la definición de una capa con el código:

```
keras.layer.Dense(512, activation='relu')
```

Esa capa se puede interpretar como una función:

``output = relu(dot(W, input) + b)``

Que toma un tensor 2D (``W``) al que le aplicamos el **producto escalar**  ``dot`` contra el tensor de entrada (``input``), que a su vez devuelve un tensor 2D que lo sumamos con un **vector** ``b`` y devuelve de nuevo un tensor 2D al que aplicamos la función ``relu.relu(x)`` que es max(x,0) lo cual devuelve un tensor 2D al ``output``

#### Operaciones elemento a elemento
- Suma y relu son elemento a elemento: Operaciones que se aplican de manera independiente a cada entrada de los tensores.
- Permiten implementaciones paralelas masivas

In [11]:
def naive_relu(x):
    assert len(x.shape) == 2 # x es un tensor 2D Numpy
    x = x.copy() # Evita sobreescribir el tensor de entrada
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

import numpy as np

x = np.array([[5, -78, 2, -34, 0],
              [6, -79, 3, -35, 1],
              [7, -80, 4, -36, 2]])


x.shape, x.shape[0], x.shape[1], naive_relu(x)

((3, 5), 3, 5, array([[5, 0, 2, 0, 0],
        [6, 0, 3, 0, 1],
        [7, 0, 4, 0, 2]]))

In [13]:
# Solo soporta suma de tensores 2D con formas idénticas
def naive_add(x, y):
    assert len(x.shape) == 2 # x e y son tensores 2D Nimpy
    assert x.shape == y.shape
    x = x.copy() # Evita sobreescribir el tensor de entrada
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

x = np.array([[5, 78, 2, 34, 0],
              [6, 79, 3, 35, 1],
              [7, 80, 4, 36, 2]])

y = np.array([[5, 78, 2, 34, 0],
              [6, 79, 3, 35, 1],
              [7, 80, 4, 36, 2]])

naive_add(x, y)

array([[ 10, 156,   4,  68,   0],
       [ 12, 158,   6,  70,   2],
       [ 14, 160,   8,  72,   4]])

Siguiendo el mismo proncípio, se pueden hacer, multiplicaciones, restas, etc, elemento a elemento. En la práctica, para el caso de matrices Numpy, estas operaciones ya están integradas y optimizadas en las **BLAS** (Basic Linear Algebra Subprograms) que están implementadas en Fortran o C. 

In [19]:
import time

x = np.random.random((20, 100))
y = np.random.random((20, 100))

t0 = time.time()
for _ in range(1000):
    z = x + y # Suma elemento a elemento
    z = np.maximum(z, 0.) # relu elemento a elemento
print("Took: {0:.2f} s".format(time.time() - t0))

Took: 0.01 s


Que es bastante más eficiente que:

In [24]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("Took: {0:.2f} s".format(time.time() - t0))

Took: 2.67 s


#### Broadcasting
- El `naive_add` implementado antes solo soporta suma de tensores 2D con formas idénticas y en ``output = relu(dot(W, input) + b)`` sumamos un tensor 2D con un vector: Si no hay "ambigüedad" el tensor más pequeño se **expandirá** para coincidir con la forma del más grande. Esto es conocido como **broadcasting** y tiene 2 pasos:
  1. Se añaden **ejes broadcast** al tensor más pequeño para que coincida con el `ndim` del tensor más grande
  2. El tensor pequeño se **repite a lo largo de los nuevos ejes** para coincidir con la forma completa del tensor grande 

In [25]:
import numpy as np
# x tiene la forma (32,10)
x = np.random.random((32, 10))
# y tiene la forma (10,)
y = np.random.random((10,))
x, y

(array([[0.31694956, 0.4443569 , 0.67207668, 0.97029237, 0.37213644,
         0.34024142, 0.60475498, 0.07518493, 0.28599376, 0.23054122],
        [0.78106453, 0.31717006, 0.59247799, 0.6410915 , 0.1913802 ,
         0.67263727, 0.36078845, 0.08022667, 0.98617988, 0.13411872],
        [0.07590996, 0.05675749, 0.0778534 , 0.40161597, 0.85360222,
         0.68056203, 0.04814399, 0.32459117, 0.63352812, 0.36707664],
        [0.74224304, 0.31451547, 0.96974088, 0.04797153, 0.87217818,
         0.58809147, 0.74288529, 0.01094437, 0.81964618, 0.66823464],
        [0.96153305, 0.26086255, 0.08511175, 0.71774906, 0.23253944,
         0.4802278 , 0.04244688, 0.71873088, 0.64575751, 0.48520886],
        [0.47226463, 0.39078865, 0.73332342, 0.5297679 , 0.86467075,
         0.29959546, 0.95749654, 0.43498836, 0.78309871, 0.84197412],
        [0.92315426, 0.7450499 , 0.47899245, 0.96761538, 0.29472065,
         0.59824568, 0.0152718 , 0.90918736, 0.83128463, 0.31670263],
        [0.22611386, 0.6456

In [26]:
# Añadimos un eje vacío a y
y = np.expand_dims(y, axis=0)
y.shape

(1, 10)

In [27]:
# Repetimos y 32 vaces a lo largo de este nuevo eje. Así que acabamos
# con un tensor Y con la forma (32, 10), donde Y[i, :]==i en range(0, 32)
Y = np.concatenate([y] * 32, axis=0)
Y.shape

(32, 10)

In [28]:
Y

array([[0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
        0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295],
       [0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
        0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295],
       [0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
        0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295],
       [0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
        0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295],
       [0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
        0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295],
       [0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
        0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295],
       [0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
        0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295],
       [0.7882653 , 0.65895926, 0.8513523

In [31]:
Y[0]

array([0.7882653 , 0.65895926, 0.85135237, 0.33048101, 0.82218908,
       0.55099704, 0.76017465, 0.93063392, 0.14507332, 0.71653295])

In [34]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

# x tiene la forma (32,10)
x = np.random.random((32, 10))
# y tiene la forma (10,)
y = np.random.random((10,))
x, y, naive_add_matrix_and_vector(x, y)

(array([[1.42253572e-01, 5.45923104e-01, 5.85925050e-02, 8.21225168e-02,
         2.17174551e-01, 1.30055887e-01, 9.74953098e-01, 7.66076654e-01,
         7.65574012e-01, 2.09099837e-01],
        [3.52762855e-01, 4.12007926e-01, 2.77151222e-01, 7.66665203e-01,
         1.45179481e-01, 1.80737186e-02, 4.14135484e-01, 6.29547044e-01,
         9.36627220e-01, 9.89455617e-01],
        [2.70379105e-01, 6.51052044e-01, 7.92349996e-01, 2.19276637e-01,
         3.04395683e-01, 4.67487998e-01, 8.67125224e-01, 2.52691768e-01,
         1.74905384e-01, 3.57727135e-01],
        [8.97145412e-01, 7.23212860e-01, 4.99110625e-01, 8.09320654e-01,
         2.85501640e-01, 1.12984663e-01, 9.14199184e-01, 1.04732605e-01,
         4.83850857e-01, 9.79790475e-01],
        [5.40002818e-01, 7.35097454e-01, 8.57826746e-01, 8.16917043e-03,
         1.80265404e-01, 4.54903818e-02, 2.02323863e-01, 3.47696442e-01,
         5.37262593e-01, 9.34172522e-01],
        [8.64316776e-01, 9.43088296e-01, 7.58591159e-01, 5.8

Broadcasting se puede aplicar a dos tensores si uno tiene la forma `(a, b, ... n, n+1, ... m)` y el otro `(n, n+1, ... m)`. El broadcasting se producirá automáticamente para los ejes `a` hasta `n-1`

In [35]:
# Ejemplo aplicación operación maximun elemento a elemento a dos tensores
# de distinta forma mediante broadcasting
import numpy as np
x = np.random.random((64, 3, 32, 10))
y = np.random.random((32, 10))
# El resultado z tendrá forma (64, 3, 32, 10), como x
z = np.maximum(x, y)
z.shape

(64, 3, 32, 10)

#### Producto Tensorial
- El `naive_add` impleme